In [1]:
%pwd

'e:\\Oslo\\OsloMet\\Fourth semester\\detectionofballotpaper_with_mlops\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
pwd

'e:\\Oslo\\OsloMet\\Fourth semester\\detectionofballotpaper_with_mlops'

In [14]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/surajacharya62/chest_ct_scan_classifier_with_mlops_dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="surajacharya62"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bae467a5cfadb6e0dee59eb7356f7ad1cc5abaf0"


In [41]:
# export MLFLOW_TRACKING_URI=https://dagshub.com/surajacharya62/chest_ct_scan_classifier_with_mlops_dvc.mlflow
# export MLFLOW_TRACKING_USERNAME=surajacharya62
# export MLFLOW_TRACKING_PASSWORD=bae467a5cfadb6e0dee59eb7356f7ad1cc5abaf0

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [7]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/data/train",
                mlflow_uri="https://dagshub.com/surajacharya62/chest_ct_scan_classifier_with_mlops_dvc.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )                 

          return eval_config
    

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [18]:
class Evaluation:

    def __init__(self, config: EvaluationConfig):
          self.config = config
            
    def _valid_generator(self):
            datagenerator_kwargs = dict(
                rescale = 1./255,
                validation_split = 0.30
            )

            dataflow_kwargs = dict(
                target_size=self.config.params_image_size[:-1], 
                batch_size=self.config.params_batch_size,
                interpolation="bilinear"
            )

            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset="validation",
                shuffle=False,
                **dataflow_kwargs
            )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
          self.model = self.load_model(self.config.path_of_model)
          self._valid_generator()
          self.score = model.evaluate(self.valid_generator)
          self.save_score()
    

    def save_score(self):
          scores = {"score": self.score[0],"accuracy":self.score[1]}
          save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        print(mlflow.get_tracking_uri(), "get tacking uri+++++++++++++++++++++++")
        # tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        # print(tracking_url_type_store, "trackinggg----------------999999999999")
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")



In [19]:
try:
    config = ConfigurationManager()
    evalu_config = config.get_evaluation_config()
    evaluation = Evaluation(evalu_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-08-15 15:38:18,114: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-15 15:38:18,119: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-15 15:38:18,120: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
5/5 [==============================] - 15s 3s/step - loss: 32.3580 - accuracy: 0.5686
[2024-08-15 15:38:33,876: INFO: common: json file saved at: scores.json]
https://dagshub.com/surajacharya62/chest_ct_scan_classifier_with_mlops_dvc.mlflow get tacking uri+++++++++++++++++++++++
[2024-08-15 15:38:33,912: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /surajacharya62/chest_ct_scan_classifier_with_mlops_dvc.mlflow/api/2.0/mlflow/runs/create]


NameError: name 'tracking_url_type_store' is not defined